In [ ]:
from datasets import load_dataset
import re
import pandas as pd

dataset = load_dataset("bilalRahib/fiqa-personal-finance-dataset")
train_dataset = dataset["train"]
df = train_dataset.to_pandas()


Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
def clean(text):
    text = text.strip().replace("\n", " ")
    text = re.sub(r"\s+", " ", text)
    return text

df["input"] = df["input"].apply(clean)
df["output"] = df["output"].apply(clean)

df.to_csv("fiqa_personal_finance_clean.csv", index=False)


In [ ]:
df.head()

,input,output
0,Does U.S. tax code call for small business own...,I am going to keep things very simple and expl...
1,What are 'business fundamentals'?,From http://financial-dictionary.thefreedictio...
2,Business Investment Loss from prior year,You need to give specific dates! In the United...
3,Would the purchase of a car for a business thr...,You don't say what country you live in. If it'...
4,Why is “cheque cashing” a legitimate business?,"In my experience (in the US), the main draw of..."


In [ ]:
from transformers import AutoTokenizer

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
    model_inputs = tokenizer(
        examples["input"],
        max_length=256,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        examples["output"],
        max_length=256,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)


In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="./finchatbot",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2
)

dataset_split = tokenized_dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)


trainer.train()


/tmp/ipython-input-1917330624.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jigyasa260604 (jigyasa260604-bennett-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,3.773700,3.193817
2,3.211300,2.844189
3,3.078800,2.804641


TrainOutput(global_step=672, training_loss=3.8374105408078147, metrics={'train_runtime': 327.1354, 'train_samples_per_second': 16.397, 'train_steps_per_second': 2.054, 'total_flos': 498558333616128.0, 'train_loss': 3.8374105408078147, 'epoch': 3.0})

In [ ]:
trainer.save_model("./finchatbot-final")
tokenizer.save_pretrained("./finchatbot-final")


('./finchatbot-final/tokenizer_config.json',
 './finchatbot-final/special_tokens_map.json',
 './finchatbot-final/spiece.model',
 './finchatbot-final/added_tokens.json',
 './finchatbot-final/tokenizer.json')

In [ ]:
from transformers import pipeline

chatbot = pipeline("text2text-generation", model="./finchatbot-final", tokenizer=tokenizer)

test_input = "What is a mutual fund?"
response = chatbot(test_input, max_length=100, do_sample=True, top_p=0.9, temperature=0.7, early_stopping=True,repetition_penalty=2.5)
print("Predicted Answer:", response[0]['generated_text'])


NameError: name 'tokenizer' is not defined

In [ ]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=abd580dca7b8a3d03815932891f73c08a1c80c16ef692d705a32c2a6a6ca5052
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
!pip install evaluate

In [ ]:
import evaluate
import pandas as pd

rouge = evaluate.load("rouge")

preds, refs = [], []

for i in range(50):
    input_text = eval_dataset[i]["input"]
    ref_text = eval_dataset[i]["output"]

    pred = chatbot(input_text, max_length=500, do_sample=False)[0]['generated_text']

    preds.append(pred)
    refs.append(ref_text)

results = rouge.compute(predictions=preds, references=refs)
print(results)


NameError: name 'eval_dataset' is not defined

In [ ]:
!pip install transformers yfinance pandas
